In [1]:
import pandas as pd
import numpy as np
import scipy as sp
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
%matplotlib inline

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [3]:
train.head()

,id,query,product_title,product_description,median_relevance,relevance_variance
0,1,bridal shower decorations,Accent Pillow with Heart Design - Red/Black,Red satin accent pillow embroidered with a hea...,1,0.000
1,2,led christmas lights,Set of 10 Battery Operated Multi LED Train Chr...,Set of 10 Battery Operated Train Christmas Lig...,4,0.000
2,4,projector,ViewSonic Pro8200 DLP Multimedia Projector,NaN,4,0.471
3,5,wine rack,Concept Housewares WR-44526 Solid-Wood Ceiling...,"Like a silent and sturdy tree, the Southern En...",4,0.000
4,7,light bulb,Wintergreen Lighting Christmas LED Light Bulb ...,"WTGR1011\nFeatures\nNickel base, 60,000 averag...",2,0.471


In [4]:
#this column with everything didn't improve kaggle score
#train['product_info'] = train['product_title'] +" "+train['product_description']+" "+train['query']
#test['product_info'] = test['product_title'] +" "+test['product_description']+" "+test['query']

In [5]:
train.shape, test.shape

((10158, 6), (22513, 4))

In [6]:
# define X and y
X_train = train['query'].fillna('none')
y = train['median_relevance']

X_test = test['query'].fillna('none')

In [7]:
# use CountVectorizer to create document-term matrices from X_train and X_test
vect = CountVectorizer(CountVectorizer(stop_words='english', ngram_range=(2,3), max_features=1000, min_df=2))
X_train_dtm = vect.fit_transform(X_train)
X_test_dtm = vect.transform(X_test)

# rows are documents, columns are terms (aka "tokens" or "features")
print(X_train_dtm.shape)
print(X_test_dtm.shape)

(10158, 487)
(22513, 487)


In [8]:
# show vectorizer options
vect

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8',
        input=CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=1000, min_df=2,
        ngram_range=(2, 3), preprocessor=None, stop_words='english',
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None),
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [9]:
#vect.fit(X_train)

In [10]:
nb = MultinomialNB()
nb.fit(X_train_dtm, y)
y_pred = nb.predict(X_test_dtm)

In [11]:
y_pred

array([4, 2, 3, ..., 1, 4, 4], dtype=int64)

In [12]:
submission = pd.DataFrame({"id": test["id"], "prediction": y_pred})
submission.to_csv("submission_may_30.csv", index=False)